In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop = nn.Dropout(0.1)
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input 1x28x28 OUtput32x28x28 RF 3x3
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1) #input 32x28x28 OUtput 64x28x28 RF 5x5
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) #input 64x28x28 OUtput 64x14x14 RF 10x10
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1) #input 64x14x14 OUtput 128x14x14 RF 12x12
        self.bn4 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1) #input 128x14x14 OUtput 256x14x14 RF 14x14
        self.bn5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1) #input 128x14x14 OUtput 256x14x14 RF 14x14
        self.bn6 = nn.BatchNorm2d(16)
        self.pool3 = nn.MaxPool2d(2, 2) #input 256x14x14 OUtput256x7x7 RF 28x28
        self.conv7 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn7 = nn.BatchNorm2d(16)
        self.conv8 = nn.Conv2d(16, 16, 3, padding=1)
        self.fc = nn.Linear(16, 10)
    
    def forward(self, x):
        x = self.pool1(self.drop(F.relu(self.bn2(self.conv2(self.drop(F.relu(self.bn1(self.conv1(x)))))))))
        x = self.pool2(self.drop(F.relu(self.bn4(self.conv4(self.drop(F.relu(self.bn3(self.conv3(x)))))))))
        x = self.pool3(self.drop(F.relu(self.bn6(self.conv6(self.drop(F.relu(self.bn5(self.conv5(x)))))))))
        x = F.adaptive_avg_pool2d(self.conv8(self.drop(F.relu(self.bn7(self.conv7(x))))), output_size=1)
        x = self.fc(x.view(-1, 16))
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
           Dropout-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           2,320
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,320
       BatchNorm2d-9           [-1, 16, 14, 14]              32
          Dropout-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
    

<ipython-input-2-cfafd98d1a79>:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.08, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-cfafd98d1a79>:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.03716764971613884 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.93it/s]



Test set: Average loss: 0.0655, Accuracy: 9809/10000 (98.09%)



loss=0.02299473248422146 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 19.96it/s]



Test set: Average loss: 0.0825, Accuracy: 9751/10000 (97.51%)



loss=0.03494829311966896 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.17it/s]


...


Test set: Average loss: 0.0302, Accuracy: 9912/10000 (99.12%)



loss=0.015650128945708275 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.13it/s]



Test set: Average loss: 0.0258, Accuracy: 9924/10000 (99.24%)



loss=0.008196117356419563 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.45it/s]



Test set: Average loss: 0.0199, Accuracy: 9939/10000 (99.39%)

